In [366]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# PEP8 and coding style

## [PEP 0: Python enhancement proposal](https://www.python.org/dev/peps/)

> This PEP contains the index of all Python Enhancement Proposals, known as PEPs. PEP numbers are assigned by the PEP editors, and once assigned are never changed [1]. The version control history [2] of the PEP texts represent their historical record.

## PEP 8 -- Style Guide for Python Code

Why do we need to write good code?
* Readability
* Less error prone
* Impress the interviewer?
* ...

### A few common mistakes
1. Variable naming. https://realpython.com/python-pep8/#naming-conventions
  * Avoid single letter names. 
  * my_variable, not myVariable. my_function, not myFunction
2. When to use whitespaces and when not to use.



```python
# wrong
foo=3
foo= 3
# correct
foo = 3


# wrong
foo = a+ 2
foo = a +2
foo = a+2

# correct
foo = a + 2


# wrong
def myfunc(arg = 3):
    pass
# correct
def myfunc(arg=3):
    pass
    
    
# wrong
foo = myfunc(arg = 3)
# corrext
foo = myfunc(arg=3)
```

```

### Linting/formatting tools
* pyflakes/flake8
* pylint
* black

# Decorators and closures

## Closure
> A closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there

In [99]:
# implementing LCG with closure
def make_lcg(m, a, c, seed):
    x = seed
    
    def inner():
        nonlocal x
        x = (a * x + c) % m
        return x
    
    return inner


lcg = make_lcg(m=2 ** 32, a=1103515245, c=12345, seed=888)
[lcg() for _ in range(10)]

UnboundLocalError: local variable 'x' referenced before assignment

In [98]:
lcg.__code__.co_varnames

()

In [12]:
lcg.__code__.co_freevars

('a', 'c', 'm', 'x')

In [317]:
lcg.__closure__

(<cell at 0x7f5406ab9f50: int object at 0x7f5406af2d70>,
 <cell at 0x7f5406ab9950: int object at 0x7f5406ad7030>,
 <cell at 0x7f5406ab94d0: int object at 0x7f5406ad7250>,
 <cell at 0x7f5406ab91d0: int object at 0x7f5406ad7230>)

In [318]:
[cell.cell_contents for cell in lcg.__closure__]

[1103515245, 12345, 4294967296, 2336383630]

## Scopes
*LEGB Rule*
1. **Local (or function) scope** is the code block or body of any Python function or lambda expression
1. **Enclosing (or nonlocal) scope** is a special scope that only exists for nested functions.
1. **Global (or module) scope** is the top-most scope in a Python program, script, or module
1. **Built-in scope** is a special Python scope that’s created or loaded whenever you run a script or open an interactive session

In [103]:
# local
def cube(base):
    result = base ** 3
    print(f'The cube of {base} is: {result}')
    return result

In [104]:
cube.__code__.co_varnames

('base', 'result')

In [45]:
cube.__code__.co_consts

(None, 3, 'The cube of ', ' is: ')

In [367]:
cube.__code__.co_freevars

()

In [107]:
# global
# dir()

In [111]:
# built-in
# dir(__builtins__)

## Decorator
A callable that takes another callable as argument

In [113]:
def logged(func):

    def inner(x):
        print(f"Running function {func.__name__} with arguments x={x}")
        return func(x)

    return inner

In [117]:
def bar(x):
    """Returns x"""
    return x

bar = logged(bar)
bar(42)

Running function bar with arguments x=42


42

In [118]:
@logged
def bar(x):
    """asdfsdf"""
    return x

bar(42)

Running function bar with arguments x=42


42

In [119]:
bar.__name__

'inner'

In [120]:
bar.__doc__

Metadata are lost!

In [121]:
import functools

def logged(func):

    @functools.wraps(func)
    def inner(x):
        print(f"Running function {func.__name__} with arguments x={x}")
        func(x)

    return inner


@logged
def bar(x):
    """Returns x"""
    return x


bar(42)

Running function bar with arguments x=42


In [122]:
bar.__doc__

'Returns x'

In [123]:
bar.__name__

'bar'

## Stacked decorators

In [125]:
def dec1(f):
    
    def inner(x):
        print(f'Running {f.__name__} with dec1')
        return f(x)
    
    return inner


def dec2(f):
    
    def inner(x):
        print(f'Running {f.__name__} with dec2')
        return f(x)
    
    return inner


@dec1
@dec2
def bar(x):
    print('calling bar')

bar(42)


def bar(x):
    print('calling bar')

dec1(dec2(bar))(42)

Running inner with dec1
Running bar with dec2
calling bar
Running inner with dec1
Running bar with dec2
calling bar


## Parameterized decorator

What if user wants to pass in the file handle where the logs will be outputed to, such as stderr?

In [126]:
import sys
import functools


def logged(file=sys.stdout):
    
    def decorate(func):

        @functools.wraps(func)
        def inner(x):
            print(f'Running {func.__name__} with dec1', file=file)
            return func(x)
    
        return inner
    
    return decorate


@logged(file=sys.stderr)
def bar(x):
    return x


bar(42)

Running bar with dec1


42

In [127]:
# equivalently
class Logged:
    
    def __init__(self, file=sys.stdout):
        self.file = file
        
    def __call__(self, func):

        @functools.wraps(func)
        def inner(x):
            print(f'Running {func.__name__} with dec1', file=self.file)
            return func(x)
    
        return inner


@Logged(file=sys.stderr)
def bar(x):
    return x


bar(42)

Running bar with dec1


42

# Iterable, iterator and generator

## Iterable and Iterator

**Python obtains iterators from iterables.**

In [28]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [222]:
s = 'ABC'

it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


Interface of an iterator: 
    
`__next__`: returns the next available item, and raise StopIteration when there are no more. 

`__iter__`: returns self. To be used in e.g. for loop

Interface of an iterable:

`__iter__`: returns an iterator. If not implemented, `__getitem__` will be used (for backward compatibility).

In [128]:
# implementing LCG with iterator

class LCG:
    
    def __init__(self, m, a, c, seed, max_iter=5):
        self.m = m
        self.a = a
        self.c = c
        self.x = seed
        self.i = 0
        self.max_iter = max_iter

    def __next__(self):
        self.x = (self.a * self.x + self.c) % self.m
        self.i += 1
        if self.i > self.max_iter:
            raise StopIteration
        return self.x

    def __iter__(self):
        return self

    
lcg = LCG(m=2 ** 32, a=1103515245, c=12345, seed=888)

for _ in range(5):
    print(next(lcg))
    
next(lcg)

669006417
4158894774
1529527223
2095963940
1696410253


StopIteration: 

In [129]:
lcg = LCG(m=2 ** 32, a=1103515245, c=12345, seed=888)

for num in lcg:
    print(num)

669006417
4158894774
1529527223
2095963940
1696410253


## Generator

In [131]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [132]:
for i in gen_123():
    print(i)

1
2
3


In [133]:
g = gen_123()
print(type(gen_123))
print(type(g))

<class 'function'>
<class 'generator'>


In [78]:
print(next(g))
print(next(g))
print(next(g))
print(next(g))


1
2
3


StopIteration: 

In [134]:
def gen_123():
    print('Returning 1')
    yield 1
    print('Returning 2')
    yield 2
    print('Returning 3')
    yield 3
    print('Finishing up')

In [135]:
g = gen_123()

In [136]:
next(g)

Returning 1


1

In [137]:
next(g)

Returning 2


2

In [138]:
next(g)

Returning 3


3

In [139]:
next(g)

Finishing up


StopIteration: 

Implementing LCG with generator

In [54]:
def lcg(m, a, c, seed):
    x = seed
    while True:
        x = (a * x + c) % m
        yield x


rand = lcg(m=2 ** 32, a=1103515245, c=12345, seed=888)
print(next(rand))
print(next(rand))
print(next(rand))
print(next(rand))
print(next(rand))

669006417
4158894774
1529527223
2095963940
1696410253


In [55]:
rand = lcg(m=2 ** 32, a=1103515245, c=12345, seed=888)
for i, x in enumerate(rand):
    if i >= 5:
        break
    print(x)

669006417
4158894774
1529527223
2095963940
1696410253


## Generators in built-in functions

In [57]:
range(100000000)

range(0, 100000000)

In [140]:
z = zip([1, 2, 3], ['a', 'b', 'c'])
print(list(z))

[(1, 'a'), (2, 'b'), (3, 'c')]


In [268]:
reversed([1, 2, 3])

In [269]:
filter(lambda x: x > 5, range(10))

# Classes

## Class vs Class instance

In [141]:
class Vehicle:
    
    can_fly = False
    num_of_wheels = 0  # class attribute


class Car(Vehicle):
        
    number_of_wheels = 4  # class attribute

    def __init__(self, color: str):
        self.color = color  # instance attribute

    def start_engine(self):
        print(f"{self.color} Vrooms!!!!!")


my_car = Car('red')
my_second_car = Car('blue')

In [142]:
type(my_car)

__main__.Car

In [144]:
isinstance(my_car, Car)

True

In [145]:
type(Car)

type

In [63]:
isinstance(my_car, Vehicle)

True

In [64]:
isinstance(Car, Car)

False

In [65]:
isinstance(Car, type)

True

In [66]:
issubclass(Car, Vehicle)

True

In [67]:
Car.number_of_wheels

4

In [68]:
my_car.number_of_wheels

4

In [69]:
my_car.start_engine()  # do not need to pass `my_car` to `self`

red Vrooms!!!!!


### Namespace

In [70]:
my_car.__dict__

{'color': 'red'}

In [147]:
Car.__dict__

mappingproxy({'__module__': '__main__',
              'number_of_wheels': 4,
              '__init__': <function __main__.Car.__init__(self, color: str)>,
              'start_engine': <function __main__.Car.start_engine(self)>,
              '__doc__': None})

In [148]:
my_car.number_of_wheels = 5

In [150]:
my_car.__dict__

{'color': 'red', 'number_of_wheels': 5}

In [151]:
my_car.number_of_wheels

5

In [152]:
Car.__dict__

mappingproxy({'__module__': '__main__',
              'number_of_wheels': 4,
              '__init__': <function __main__.Car.__init__(self, color: str)>,
              'start_engine': <function __main__.Car.start_engine(self)>,
              '__doc__': None})

### `__getattr__`/`__setattr__`

In [158]:
class Vehicle:
    
    can_fly = False
    num_of_wheels = 0  # class attribute


class Car(Vehicle):
        
    number_of_wheels = 4  # class attribute

    def __init__(self, color: str):
        self.color = color  # instance attribute

    def start_engine(self):
        print(f"{self.color} Vrooms!!!!!")

#     def __getattr__(self, attr):
#         print(f"you're trying to access attribute {attr} which is not there!")
#         return 42

    def __setattr__(self, attr, val):
        print(f"You're setting {attr} to {val}")
        super().__setattr__(attr, val)


my_car = Car('red')
# my_car.color
my_car.price = 50000
# my_car.color

You're setting color to red
You're setting price to 50000


In [159]:
my_car.__dict__

{'color': 'red', 'price': 50000}

### `__init__`/`__new__`

In [160]:
class Foo:
    
    def __new__(cls, *args):
        print(f"{cls}.__new__ called")
        return super().__new__(cls)

    def __init__(self, val):
        print(f"{self}.__init__ called")
        self.val = val
        

foo = Foo(3)

<class '__main__.Foo'>.__new__ called
<__main__.Foo object at 0x1240f8a90>.__init__ called


In [164]:
class Bar:

    def __new__(cls, *args):
        print(f"{cls.__name__}.__new__ called")
        return "SPAM"

    def __init__(self, val):
        print(f"{self.__class__.__name__}.__init__ called")
        self.val = val
        

bar = Bar(3)
bar

Bar.__new__ called


'SPAM'

In [165]:
type(bar)

str

In [162]:
class MyWeirdInt(int):

    def __new__(cls, val):
        return super().__new__(cls, val * 2)


In [163]:
MyWeirdInt(3)

6

## methods

### binding behavior: bound vs. unbound methods

Binding: `x.f(1) => f(x, 1)`

In [8]:
def bind(obj):
    
    def decorate(func):

        def func(*args, **kwargs):
            return func(obj, *args, **kwargs)
    
        return func
    
    return decorate



In [168]:
my_car.start_engine

<bound method Car.start_engine of <__main__.Car object at 0x124ae2bd0>>

In [42]:
my_car.start_engine.__func__

<function __main__.Car.start_engine(self)>

In [43]:
Car.start_engine

<function __main__.Car.start_engine(self)>

In [118]:
my_car.start_engine()

red Vrooms!!!!!


In [119]:
Car.start_engine(my_car)

red Vrooms!!!!!


### class methods and static methods

* classmethod can be useful in defining alternative constructors
* staticmethod are less useful. It can almost always be replaced with a regular function.

In [86]:
class Demo:
        
    def meth(*args):
        return args

    @classmethod
    def classmeth(*args):
        return args

    @staticmethod
    def staticmeth(*args):
        return args


d = Demo()
print(d.meth('param'))
print(d.classmeth('param'))
print(d.staticmeth('param'))

print()
print(Demo.meth('param'))
print(Demo.classmeth('param'))
print(Demo.staticmeth('param'))

(<__main__.Demo object at 0x12659d650>, 'param')
(<class '__main__.Demo'>, 'param')
('param',)

('param',)
(<class '__main__.Demo'>, 'param')
('param',)


In [169]:
class Date:
    
    def __init__(self, year, month, date):
        self.year = year
        self.month = month
        self.date = date
        
    def __repr__(self):
        return f"{self.year}/{self.month:02}/{self.date:02}"

    @classmethod
    def from_yymmdd(cls, yymmdd):
        return cls(int(yymmdd[:4]), int(yymmdd[4:6]), int(yymmdd[6:8]))


print(Date(2020, 1, 2))
print(Date.from_yymmdd('20200102'))

2020/01/02
2020/01/02


### property decorator

getter/setter implementation in Python

In [170]:
# Making Getters and Setter methods
class Celsius:
    def __init__(self, temperature = 0):
        self.set_temperature(temperature)

    def to_fahrenheit(self):
        return (self.get_temperature() * 1.8) + 32

    # getter method
    def get_temperature(self):
        return self._temperature

    # setter method
    def set_temperature(self, value):
        if value < -273.15:
            raise ValueError("Temperature below -273.15 is not possible.")
        self._temperature = value


temp = Celsius(90)
print(temp.get_temperature())

temp.set_temperature(100)
print(temp.get_temperature())

temp.set_temperature(-300)

90
100


ValueError: Temperature below -273.15 is not possible.

In [175]:
class Celsius:
    def __init__(self, temperature = 0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.get_temperature() * 1.8) + 32

    @property
    def temperature(self):
        print("getting temperature")
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        if value < -273.15:
            raise ValueError("Temperature below -273.15 is not possible.")
        self._temperature = value


temp = Celsius(90)
print(temp.temperature)

temp.temperature = 100
print(temp.temperature)

temp.temperature = -300

getting temperature
90
getting temperature
100


ValueError: Temperature below -273.15 is not possible.

### "Public" vs. "Private" attributes

In [3]:
class Demo:
    
    def __init__(self, x):
        self._x = x

    def do_something(self):
        """This is a public method"""
        self._private_method()

    def _private_method(self):
        """As a convention, prefix method with single underscore to indicate it is a private method"""

    def __private_method_with_two_leading_underscores(self):
        """Do not do this. Name mangling is confusing"""


# "name mangling"
assert '_Demo__private_method_with_two_leading_underscores' in Demo.__dict__

## Descriptor (advanced topic)
To understand more details: https://realpython.com/python-descriptors/#how-attributes-are-accessed-with-the-lookup-chain

### Duck Typing
“If it walks like a duck, and it quacks like a duck, then it must be a duck.” 

### Descriptor protocol

```python
__get__(self, obj, type=None) -> object
__set__(self, obj, value) -> None
__delete__(self, obj) -> None
```

* If an object has `__get__()`, then it’s said to be a non-data descriptor.
* If an object implements `__set__()` or `__delete__()`, then it’s said to be a data descriptor

In [180]:
class VerboseAttribute:
    def __get__(self, obj, type=None) -> object:
        print("accessing the attribute to get the value")
        return 42

    def __set__(self, obj, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")


class Foo():
    attribute1 = VerboseAttribute()
    
    def __init__(self):
        self.attribute3 = 3

    attribute2 = 2
    

my_foo_object = Foo()

In [178]:
my_foo_object.attribute1

accessing the attribute to get the value


42

In [11]:
my_foo_object.__dict__

{'attribute3': 3}

In [16]:
my_foo_object.attribute2

2

In [17]:
my_foo_object.__dict__

{'attribute3': 3}

In [18]:
my_foo_object.__class__.__dict__

mappingproxy({'__module__': '__main__',
              'attribute1': <__main__.VerboseAttribute at 0x1285be6d0>,
              '__init__': <function __main__.Foo.__init__(self)>,
              'attribute2': 2,
              '__dict__': <attribute '__dict__' of 'Foo' objects>,
              '__weakref__': <attribute '__weakref__' of 'Foo' objects>,
              '__doc__': None})

In [19]:
my_foo_object.attribute1

accessing the attribute to get the value


42

In [181]:
my_foo_object.attribute1 = 50

accessing the attribute to set the value


AttributeError: Cannot change the value

### Lookup chain
0. `__getattribute__`
1. `__get__` method of the data descriptor
2. `__dict__` of the instance
3. `__get__` method of the non-data descriptor
4. `__dict__` of the class (type)
5. `__dict__` of the parent class, or the parent's parent class, ...
6. raise AttributeError, and call `__getattr__` if it exists.


### @property is a descriptor

In [182]:
class Demo:
    
    def __init__(self, x):
        self._x = x

    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, val):
        self._x = val
    

class Demo:
    
    def __init__(self, x):
        self._x = x

    def get_x(self) -> object:
        return self._x
    
    def set_x(self, val) -> object:
        self._x = val

    x = property(get_x, set_x)  # implements descriptor protocol



obj = Demo(42)
print(obj.x)
obj.x = 43
print(obj.x)

42
43


In [183]:
Demo.x.__get__

<method-wrapper '__get__' of property object at 0x1242f4710>

In [184]:
Demo.x.__set__

<method-wrapper '__set__' of property object at 0x1242f4710>

### Method is a descriptor

In [34]:
import types

class Function(object):
    ...
    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        if obj is None:
            return self
        return types.MethodType(self, obj)  # types.MethodType binds obj with self

In [187]:
class Foo:

    def bar(self):
        pass
    

In [191]:
import types

In [189]:
foo = Foo()
foo.bar

<bound method Foo.bar of <__main__.Foo object at 0x124183c10>>

In [39]:
types.MethodType(int, 3)()

3

In [31]:
class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"
    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f

In [196]:
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"
    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        def newfunc(*args):
            return self.f(klass, *args)
        return newfunc

## Inheritance

### super()

In [127]:
class Animal:
    def __init__(self, name):
        print(name, 'is an animal.');
        
    def __repr__(self):
        return f"{self.__class__.__name__}"


class Mammal(Animal):
    def __init__(self, name):
        print(name, 'is a warm-blooded animal.')
        super().__init__(name)


class Dog(Mammal):
    def __init__(self):
        print('Dog has 4 legs.');
        super().__init__(self, 'Dog')


dog = Dog()

Dog has 4 legs.
Dog is a warm-blooded animal.
Dog is an animal.


### Multiple-inheritance and MRO

MRO: method resolution order

In [6]:
class Animal:
    def __init__(self, name):
        print(name, 'is an animal.');
        
    def __repr__(self):
        return f"{self.__class__.__name__}"

class Mammal(Animal):
    def __init__(self, name):
        print(name, 'is a warm-blooded animal.')
        super().__init__(name)

class NonWingedMammal(Mammal):
    def __init__(self, name):
        print(name, "can't fly.")
        super().__init__(name)

class NonMarineMammal(Mammal):
    def __init__(self, name):
        print(name, "can't swim.")
        super().__init__(name)

class Dog(NonWingedMammal, NonMarineMammal):
    def __init__(self):
        print('Dog has 4 legs.');
        super().__init__('Dog')

dog = Dog()
print('')
bat = NonMarineMammal('Bat')

Dog has 4 legs.
Dog can't fly.
Dog can't swim.
Dog is a warm-blooded animal.
Dog is an animal.

Bat can't swim.
Bat is a warm-blooded animal.
Bat is an animal.


In [5]:
Dog.__mro__

(__main__.Dog,
 __main__.NonWingedMammal,
 __main__.NonMarineMammal,
 __main__.Mammal,
 __main__.Animal,
 object)

In [28]:
super(NonMarineMammal, dog).__init__('dog')

dog can't fly.
dog is a warm-blooded animal.
dog is an animal.


### Subclassing built-in types

In [202]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)


dd = DoppelDict(one=1)
print(dd)

{'one': 1}


In [203]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [204]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [205]:
from collections import UserDict

class DoppelDict(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)



dd = DoppelDict(one=1)
print(dd)

{'one': [1, 1]}


In [206]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [207]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

## Saving memory footprint with __slots__

In [148]:
class Vector2d:

    __slots__ = ['_x', '_y']

v = Vector2d()

v.__dict__

AttributeError: 'Vector2d' object has no attribute '__dict__'

In [208]:
# cannot have other attributes not specified in __slots__
# this is a side effect, and should not be used to prevent users from creating new attributes

class Vector2d:
    
    __slots__ = ['_x', '_y']

    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

Vector2d(1, 2, 3)

AttributeError: 'Vector2d' object has no attribute '_z'

# Context Manager

## Review try/except/else/finally

In [209]:
try:
    1 + 0
except ZeroDivisionError:
    print('caught exception')
else:
    print('no exception caught')
finally:
    print('finally')

no exception caught
finally


In [181]:
try:
    1 / 0
except ZeroDivisionError:
    print('caught exception')
else:
    print('no exception caught')
finally:
    print('finally')

caught exception
finally


## "with" block

Simplifies try/finally

In [212]:
!echo hello > /tmp/test.txt

In [213]:
try:
    fp = open('/tmp/test.txt')
    print(fp.read())
finally:
    fp.close()

print(fp.closed)
print(fp.read())

hello

True


ValueError: I/O operation on closed file.

In [214]:
# written with "with" block
with open('/tmp/test.txt') as fp:  # fp is bound to the opened file because the file’s __enter__ method returns self.
    fp.read()

# The fp variable is still available
print(fp.closed)
# But you can’t perform I/O with fp because at the end of the with block, 
# the TextIOWrapper.__exit__ method is called and closes the file.
print(fp.read())  

True


ValueError: I/O operation on closed file.

## `__enter__` and `__exit__`

In [215]:
class LookingGlass:
    
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'RETURN VALUE OF __enter__'
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
        
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True  #  tell the interpreter that the exception was handled.

In [217]:
with LookingGlass() as what:
    print('Welcome to the Python workshop')
    print(what)

print()
print(what)
print('Back to normal.')
    

pohskrow nohtyP eht ot emocleW
__retne__ FO EULAV NRUTER

RETURN VALUE OF __enter__
Back to normal.


In [218]:
with LookingGlass() as what:
    1 / 0
    print(what)

print()
print(what)
print('Back to normal.')

Please DO NOT divide by zero!

RETURN VALUE OF __enter__
Back to normal.


In [220]:
with LookingGlass() as what:
    raise ValueError("asdfsdf")

ValueError: asdfsdf

In [221]:
print()
print(what)
print('Back to normal.')


RETURN VALUE OF __enter__
Back to normal.


## contextlib

In [222]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    yield 'RETURN VALUE OF yield'

    sys.stdout.write = original_write


with looking_glass() as what:
    print('Welcome to the Python workshop')
    print(what)
    
print()
print(what)
print("Back to normal")

pohskrow nohtyP eht ot emocleW
dleiy FO EULAV NRUTER

RETURN VALUE OF yield
Back to normal


How to handle exception?

In [223]:
with looking_glass() as what:
    1 / 0


ZeroDivisionError: division by zero

In [224]:
print('Whoops!')

!spoohW


In [1]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'RETURN VALUE OF yield'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

with looking_glass() as what:
    raise ValueError()
    print(what)


ValueError: 

In [2]:
print(what)
print("Back to normal")

RETURN VALUE OF yield
Back to normal
